### Programa que ajusta períodos "instantáneos" usando el .polycos de una observación

*Nota*: en el archivo original se usaba la notacion científica con "D", cambiarla a "e".

Importamos los paquetes que vamos a usar

In [1]:
import matplotlib.mlab as mlab
import matplotlib as mpl
import matplotlib.pyplot as plt     # para graficar

import math
import decimal
import numpy as np
import glob                         # para hacer listas de archivos

Función que calcula la frecuencia y el período a partir de los parámetros 

In [2]:
def PERIOD(T, F0, COEFF, TMID):
    
    DT = (T - TMID) * 1440.
    
    FREQ = ( 11*(DT**10)*COEFF[11] + 10*(DT**9)*COEFF[10] + 9*(DT**8)*COEFF[9] + 8*(DT**7)*COEFF[8] + 7*(DT**6)*COEFF[7] + 6*(DT**5)*COEFF[6] + 5*(DT**4)*COEFF[5] + 4*(DT**3)*COEFF[4] + 3*(DT**2)*COEFF[3] + 2*DT*COEFF[2] + COEFF[1] ) * (1/60.) + F0
        
    return 1/FREQ

Abrimos el archivo .polycos y el archivo de salida

In [3]:
file = glob.glob('*.polycos')[0]      # buscamos el archivo .polycos
print(file)
lines=sum(1 for line in open(file))   # leemos el número de líneas
out = open("periods.dat", "w+")       # abrimos el archivo de salida

prepfold_timing_20200318_201231_PSR_0835-4510.pfd.polycos


Arreglos que vamos a usar

In [4]:
ncoef = np.genfromtxt ( file, comments="none", dtype=int, skip_header=1, max_rows=1, usecols=(4) ) # leemos el número de coeficientes
COEFF = np.zeros(ncoef)   # vector de coeficientes

En el archivo .polycos la información está en bloques de datos, cada uno correspondiente a un MJD distinto.
Dentro de cada bloque tenemos:
1. Una línea con el nombre de púlsar, fecha, UTC, TMID, DM.
2. Una línea con RPHASE, F0, observatorio, duración, número de coeficientes, frecuencia de observación.
3. Cuatro líneas con los coeficientes, con tres coeficientes por línea (12 en total).

In [7]:
T = 58924.65583333330

n = 0               # número de líneas leídas del archivo .polycos
not_found = True

while not_found:

    TMID = np.genfromtxt( file, comments="none", dtype=np.float128, skip_header=n, max_rows=1, usecols=(3) ) # Middle point of the time span in MJD  
    span_min = np.genfromtxt( file, comments="none", dtype=float, skip_header=n+1, max_rows=1, usecols=(3))  # Time span in minutes
    span_mjd = span_min * (1./(60.*24.))                                                                     # Time span in MJD
        
    T_inicial = TMID - span_mjd/2.                                                                           # Starting point of the time span in MJD
    T_final   = TMID + span_mjd/2.

    if T >= T_inicial and T < T_final:
    
        F0 = np.genfromtxt( file, comments="none", dtype=float, skip_header=n+1 , max_rows=1, usecols=(1) )      # Reference spin frequency
        matrix = np.genfromtxt(file, usecols=range(3), dtype=np.float128, skip_header=n+2, max_rows=4)           # Matriz de coeficientes
        COEFF = matrix.flatten('C')                                                                              # Vector de coeficientes

        P_seg = PERIOD(T, F0, COEFF, TMID)   # período en segundos
#        print("Entre MJD " + str(T_inicial) + " y " + str(T_final))
#        print("Correspodiente a TMID = " + str(TMID))
        print("P[ms] = " + str(P_seg * 1000.))
        
        not_found = False
        
    else:
    
        n += 6
        
        if n >= lines:
            print("ERROR: el T indicado no se encuentra dentro del .polycos")
            return

Entre MJD 58924.62499999996667 y 58924.666666666633333
Correspodiente a TMID = 58924.6458333333
P[ms] = 89.408068395468960655
